In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd

In [3]:
conf_dw = {
      'url':'jdbc:mysql://localhost:3306/etlmysql?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigMysql',
      'password':'bigMysql1234@'   
      }
}

conf_dm = {
      'url':'jdbc:mysql://localhost:3306/etlmysqlDM?characterEncoding=utf8&serverTimezone=Asia/Seoul'
     ,'props':{
      'user':'bigDM',
      'password':'bigDM1234@'   
      }
}

In [4]:
# table 저장 / 반환 관련한 함수 구성
# config : db 연결 정보
# spark.read.jdbc() : db에서 명시된 테이블을 반환받는 모듈
def find_data(config, table_name) :
    return spark.read.jdbc(url= config['url'], table=table_name, properties=config['props'])

def save_data(config, df, table_name) :
    return df.write.jdbc(url= config['url'], table=table_name, mode='overwrite' , properties=config['props'])

In [6]:
food_info = find_data(conf_dw, 'FOOD_INFO')

In [7]:
food_info.show()
type(food_info)

+------------+----------------------------------+------------------------------------+
|        food|                              info|                          ingredient|
+------------+----------------------------------+------------------------------------+
|    비빔냉면|    냉면(冷麵, 문화어: 랭면)은 ...|    냉면사리 2인분,시판냉면육수 1...|
|      비빔밥|  비빔밥은 한국의 밥 요리이다. ...|         돼지고기 100g,애호박 1/5...|
|      삼겹살| 삼겹살의 다른 뜻은 다음과 같다...|         돼지고기 600g,깻잎 10장,...|
|      삼계탕|  삼계탕(蔘鷄湯, 문화어: 인삼닭...|    닭 2마리,삼계탕용약재,찹쌀,마...|
|      설렁탕|         설렁탕(영어: Seolleong...|          무 1개,쪽파 150g,설탕 4...|
|        순대| 순대는 한국의 음식으로, 돼지의...|        순대500g,양배추3컵,깻잎 1...|
|  순두부찌개|순두부찌개는 순두부를 주재료로 ...|        순두부,대파 1/4개,양파 1/...|
|    양념게장|    양념게장(---醬)은 한국의 게...|     냉동 꽃게 4마리,간장(소주컵)...|
|    양념치킨|양념치킨은 한국식 닭튀김 요리로...|      물엿 4큰술,다진 마늘 2큰술,...|
|        육회|육회는 대한민국의 요리에서 회의...|     소고기(앞다리/우둔/엉덩이) 1...|
|        잡채|        잡채(雜菜, 영어: Japcha...|    당면 1줌반,당근 약간,파프리카...|
|    제육볶음|        제육 볶음(豬肉-, Stir-f...|    돼

pyspark.sql.dataframe.DataFrame

In [8]:
pork = ['돼지', '족발', '햄']
beef = ['소고기','소불고기']
egg = ['계란','달걀']
offal = ['곱창','순대']
def categorize_ingredient(ingredient):
    for e in pork:
        if e in ingredient:
            return 'Pork'
    for e in beef:
        if e in ingredient:
            return 'Beef'
    for e in egg:
        if e in ingredient:
            return 'Egg'
    for e in offal:
        if e in ingredient:
            return 'Offal'
    else:
        return 'None'

In [10]:
# 사용자 정의 함수를 PySpark UDF로 변환
categorize_udf = udf(categorize_ingredient, StringType())

# 'taboo_ingredient' 컬럼 추가
result_df =food_info.withColumn('taboo_ingredient', categorize_udf('ingredient'))

# 결과 확인
result_df.show()

+------------+----------------------------------+------------------------------------+----------------+
|        food|                              info|                          ingredient|taboo_ingredient|
+------------+----------------------------------+------------------------------------+----------------+
|    비빔냉면|    냉면(冷麵, 문화어: 랭면)은 ...|    냉면사리 2인분,시판냉면육수 1...|             Egg|
|      비빔밥|  비빔밥은 한국의 밥 요리이다. ...|         돼지고기 100g,애호박 1/5...|            Pork|
|      삼겹살| 삼겹살의 다른 뜻은 다음과 같다...|         돼지고기 600g,깻잎 10장,...|            Pork|
|      삼계탕|  삼계탕(蔘鷄湯, 문화어: 인삼닭...|    닭 2마리,삼계탕용약재,찹쌀,마...|            None|
|      설렁탕|         설렁탕(영어: Seolleong...|          무 1개,쪽파 150g,설탕 4...|            None|
|        순대| 순대는 한국의 음식으로, 돼지의...|        순대500g,양배추3컵,깻잎 1...|           Offal|
|  순두부찌개|순두부찌개는 순두부를 주재료로 ...|        순두부,대파 1/4개,양파 1/...|             Egg|
|    양념게장|    양념게장(---醬)은 한국의 게...|     냉동 꽃게 4마리,간장(소주컵)...|            None|
|    양념치킨|양념치킨은 한국식 닭튀김 요리로...|      물엿 4큰술

In [13]:
save_data(conf_dm, result_df, 'FOOD_INFO_WITH_TABOO')

In [5]:
restaurant_list = find_data(conf_dw, 'RESTAURANT_LIST')

In [7]:
# 크롤링한 데이터에서 서울이 아닌 다른지역 음식점 데이터 제거
restaurant_list = restaurant_list.filter(col("addr").startswith("서울"))
restaurant_list.count()
restaurant_list.printSchema()

2070

root
 |-- addr: string (nullable = true)
 |-- food: string (nullable = true)
 |-- re_rank1: string (nullable = true)
 |-- re_rank2: string (nullable = true)
 |-- re_rank3: string (nullable = true)
 |-- re_rank4: string (nullable = true)
 |-- re_rank5: string (nullable = true)
 |-- re_visitor: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- tel: string (nullable = true)



In [11]:
df = restaurant_list

In [8]:
# 카카오 API KEY
API_KEY = "98614acfa7370dcb5a0d9e815d6c9e5b"

In [9]:
def convert_address_to_coordinates(address):
    api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {API_KEY}"}

    params = {"query": address}
    
    try:
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()
        result = response.json()

        if "documents" in result and len(result["documents"]) > 0:
            coordinates = result["documents"][0]["y"], result["documents"][0]["x"]
            return coordinates
        else:
            return None
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

In [10]:
def add_coordinates_to_dataframe(df, address_column):
    latitudes = []
    longitudes = []

    for address in df[address_column]:
        coordinates = convert_address_to_coordinates(address)
        if coordinates:
            latitudes.append(coordinates[0])
            longitudes.append(coordinates[1])
        else:
            latitudes.append(None)
            longitudes.append(None)

    df["decimalLatitude"] = latitudes
    df["decimalLongitude"] = longitudes

In [12]:
add_coordinates_to_dataframe(df, 'addr')

TypeError: Column is not iterable